In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from joblib import load, dump

from src.scripts.artemis_data import get_ns_cols, get_email_cols

In [68]:
df = pd.read_csv("../data/processed/final_whois_data.csv")

In [69]:
label = 'community'
y = df[label]
X = df.drop(columns=label)

In [70]:
enc_dict = load("../models/enc_dict.joblib")
country_encoder = load('../models/country_encoder.joblib')

In [71]:
cols_to_drop = ['domain', 'updated_date', 'expiration_date', 'creation_date', 'days_since_creation', 'malicious']
cols_to_drop += get_ns_cols(7)
cols_to_drop += get_email_cols(3)
X = X.drop(columns=cols_to_drop)


In [72]:
for col in enc_dict.keys():
    X[col] = enc_dict[col].transform(X[col])
for col in ['country', 'dns_rec_a_cc', 'dns_rec_mx_cc']:
    X[col] = country_encoder.transform(X[col])

In [73]:
X = X.fillna(-1)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [80]:
clf = KNeighborsClassifier(weights='distance').fit(X_train, y_train)

In [81]:
clf.score(X_test, y_test)

0.8070921985815603

In [84]:
dump(clf, '../models/community_predictor.joblib')

['../models/community_predictor.joblib']